<a href="https://colab.research.google.com/github/AlessandroBarbiero/deep-learning-for-games-labs/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo Tree Search Lab

In this lab, we'll be using the game connect four, as a vehicle for learning MinMax and Monte Carlo Tree Search.
We'll also introduce concepts, such as state, that'll stay relevant throughout the course.
Expect to lose in connect four to the algorithm at the end of the lab.

## Setup
This section you won't need to edit, but it is worth skimming through—this is where we declare the objects you'll be interacting with througout the lab.

In [1]:
# imports
import random
from typing import List, Tuple
import time
from copy import deepcopy # world -> thought

In [28]:
# world and world model
class State:
    def __init__(self, cols=7, rows=6, win_req=4):
        self.board = [['.'] * cols for _ in range(rows)]
        self.heights = [1] * cols
        self.num_moves = 0
        self.win_req = win_req

    def get_avail_actions(self) -> List[int]:
        return [i for i in range(len(self.board[0])) if self.heights[i] <= len(self.board)]
  
    def put_action(self, action, agent):
        self.board[len(self.board) - self.heights[action]][action] = agent.name
        self.heights[action] += 1
        self.num_moves += 1

    def is_over(self):
        return self.num_moves >= len(self.board) * len(self.board[0])

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        header  = " ".join([str(i) for i in range(len(self.board[0]))])
        line    = "".join(["-" for _ in range(len(header))])
        board   = [[e for e in row] for row in self.board]
        board   = '\n'.join([' '.join(row) for row in board])
        return  '\n' + header + '\n' + line + '\n' + board + '\n'


In [3]:
def diags_pos(board, n_rows, n_cols):
    """Get positive diagonals, going from bottom-left to top-right."""
    for di in ([(j, i - j) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
        yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

def diags_neg(board, n_rows, n_cols):
    """Get negative diagonals, going from top-left to bottom-right."""
    for di in ([(j, i - n_cols + j + 1) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
        yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

# evaluate the utility of a state
def utility(state: 'State'):
    board = state.board
    n_cols = len(board[0]) - 1
    n_rows = len(board) - 1

    cols = list(map(list, list(zip(*board))))
    rows = board
    diags = list(diags_neg(board, n_rows, n_cols)) + list(diags_pos(board, n_rows, n_cols))
    lines = rows + cols + diags
    strings = ["".join(s) for s in lines]
    for string in strings:
        if 'OOOO' in string:
            return -1
        if 'XXXX' in string:
            return 1
    return 0


In [4]:
# parent class for mcts, minmax, human, and any other idea for an agent you have
class Agent:
    def __init__(self, name: str):
        self.name: str = name

    def get_action(self, state: State):
        return random.choice(state.get_avail_actions())

In [31]:
# connecting states and agents
class Game:
    def __init__(self, agents: Tuple[Agent], state = None):
        self.agents = agents
        if state == None:
            self.state = State()
        else:
            self.state = state
            # considering that the agent O will always start, reorder it if not write
            if state.num_moves % 2 == 0 and agents[0].name == 'X':
                self.agents = (agents[1], agents[0]) 

    def play(self, moves: int = -1) -> int:
        if moves < 0:
            while utility(self.state) == 0 and not self.state.is_over():
                for agent in self.agents:
                    if utility(self.state) == 0 and not self.state.is_over():
                        action = agent.get_action(self.state)
                        self.state.put_action(action, agent)
                        print(self.state)
        else:
            while utility(self.state) == 0 and not self.state.is_over() and moves > 0:
                for agent in self.agents:
                    if utility(self.state) == 0 and not self.state.is_over():
                        action = agent.get_action(self.state)
                        self.state.put_action(action, agent)
                    #    print(self.state)
                moves -= 1
        return utility(self.state)

## Exercise 0: Discuss and Run game
Let's discuss if the `utility` function best belongs to the state or the agent.
Put the state, agent and game class together so that a game is run.

I would say that the utility function best belongs to the agent, because each agent can evaluate the utility of a state following a different strategy.

In [40]:
agents = (Agent('O'), Agent('X'))
game = Game(agents)
value = game.play()
print(value)


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . O . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . O X O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
X . . . O X O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . O . .
X . . . O X O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . O . .
X . . . O X O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . O . .
X X . . O X O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X .

## Exercise 1: Human Agent
Make a child class of `Agent` called `Human`, with the `get_action` method overwritten to take input from you. *hint*: use `int(input())`

In [54]:
class Human(Agent):
  def __init__(self, name):
    super(Human, self).__init__(name)

  def get_action(self, state: State):
    print("Select the column you want to insert your tile to")
    action = int(input())
    while action not in state.get_avail_actions():
      print("Action not available, retry")
      action = int(input())
    return action
      

## Exercise 2: Gekko
Make a child class of `Agent` called `Gekko`, with a `get_action` that is very short sighted (greedy). You can basically do whatever you want here, as long as your output a valid action. You might want to make a `utility` function for the agent, and perhaps some helper functions. Write a two line comment explaining your Gekko's heuristic.

In [23]:
class Gekko(Agent):
  def __init__(self, name):
    super(Gekko, self).__init__(name)

  ''' Return the number of adjacent tiles of your tipe after a certain action is completed '''
  def utility(self, state:State, action:int):
    if action not in state.get_avail_actions():
      return -10
    board = state.board
    n_cols = len(board[0]) - 1
    n_rows = len(board) - 1

    utility_row = 1
    utility_col = 1
    utility_diag_pos = 1
    utility_diag_neg = 1

    row_pos = len(board) - state.heights[action]
    # calculate utility for columns
    for i in range(row_pos+1, n_rows+1):
      if board[i][action] == self.name:
        utility_col += 1
      else:
        break

    # caluclate utility for rows
    for i in range(action-1, -1, -1):
      if board[row_pos][i] == self.name:
        utility_row += 1
      else:
        break
    for i in range(action+1, n_cols+1):
      if board[row_pos][i] == self.name:
        utility_row += 1
      else:
        break

    # Calculate utility for positive diagonal
    for i, j in zip(range(row_pos+1, n_rows+1), range(action-1, -1, -1)):
      if board[i][j] == self.name:
        utility_diag_pos += 1
      else:
        break
    for i, j in zip(range(row_pos-1, -1, -1), range(action+1, n_cols+1)):
      if board[i][j] == self.name:
        utility_diag_pos += 1
      else:
        break

    # Calculate utility for negative diagonal
    for i, j in zip(range(row_pos-1, -1, -1), range(action-1, -1, -1)):
      if board[i][j] == self.name:
        utility_diag_neg += 1
      else:
        break
    for i, j in zip(range(row_pos+1, n_rows+1), range(action+1, n_cols+1)):
      if board[i][j] == self.name:
        utility_diag_neg += 1
      else:
        break

 #   print("The utility values for action", action, "are:\nUtility col->", utility_col, "\nUtility row->", utility_row, "\nUtility positive diagonal->", utility_diag_pos, "\nUtility negative diagonal->", utility_diag_neg,)
    ut = max(utility_col, utility_row, utility_diag_pos, utility_diag_neg)
    print("The utility function for action", action, "is:", ut)
    return ut


  ''' Select the action that generate more consecutive tiles '''
  def get_action(self, state: State):
    chosen_action = 0
    chosen_utility = 0
    for action in state.get_avail_actions():
      utility = self.utility(state, action)
      if utility > chosen_utility:
        chosen_action = action
        chosen_utility = utility
    return chosen_action


In [10]:
agents = (Gekko('O'), Human('X'))
game = Game(agents)
game.play()

The utility function for action 0 is: 1
The utility function for action 1 is: 1
The utility function for action 2 is: 1
The utility function for action 3 is: 1
The utility function for action 4 is: 1
The utility function for action 5 is: 1
The utility function for action 6 is: 1

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .

Select the column you want to insert your tile to
6

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . X

The utility function for action 0 is: 2
The utility function for action 1 is: 2
The utility function for action 2 is: 1
The utility function for action 3 is: 1
The utility function for action 4 is: 1
The utility function for action 5 is: 1
The utility function for action 6 is: 1

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . X

Select the column you want to insert your tile

## *Optional exercise: MinMax (useful to have done for exercise 3)*
Make a MinMax agent:

In [24]:
class MinMax(Agent):
  def __init__(self, name):
    super(MinMax, self).__init__(name)

  def get_action(self, state: State):
    best_action = -1
    maximizing = True
    best_score = -10
    for action in state.get_avail_actions():
      state_copy = deepcopy(state)
      state_copy.put_action(action)
      score = self.minimax(state_copy, not maximizing)
      if score > best_score:
        best_action = action
        best_score = score
    
    return best_action

  def minimax(self, state, maximizing):
    if state.is_over() and maximizing:
      return 1
    elif state.is_over() and not maximizing:
      return -1

    elif maximizing:
      best_score = -10
      for action in state.get_avail_actions():
        state_copy = deepcopy(state)
        state_copy.put_action(action)
        score = self.minimax(state_copy, not maximizing)
        if score > best_score:
          best_score = score
      return best_score

    elif not maximizing:
      best_score = 10
      for action in state.get_avail_actions():
        state_copy = deepcopy(state)
        state_copy.put_action(action)
        score = self.minimax(state_copy, not maximizing)
        if score < best_score:
          best_score = score
      return best_score
    else:
      return 0
      

In [ ]:
agents = (MinMax('O'), Human('X'))
game = Game(agents)
game.play()

## Exercise 3: MCTS
Same but for Monte Carlo Tree Search. See if you can beat it with a `Human`.

In [45]:
class Node:
    def __init__(self, state: State, parent: 'Node' = None, action = None):
        self.children: List['Node'] = []
        self.parent: 'Node' = parent
        self.state: State = state
        self.value: int = 0
        self.action: int = action

    def expand(self, agent):
        for action in self.state.get_avail_actions():
            state_copy = deepcopy(self.state)
            state_copy.put_action(action, agent)
            self.children.append(Node(state_copy, self, action))
    
# Suppose that O agent always plays first
    def rnd_evaluate(self, name, moves = 10):
        agents = (Agent('O'), Agent('X'))
        game = Game(agents, self.state)
        if name == 'O':
            self.value = -game.play(moves)
        else:
            self.value = game.play(moves)
    
    def backpropagate(self):
        self.parent.value += self.value


In [50]:
''' Each turn it computes two levels of the tree and after that plays a 
bunch of random games for each state backpropagating then the chance of win'''
class MCTS(Agent):
    def __init__(self, name):
        super(MCTS, self).__init__(name)
        
    def get_action(self, state: State):
        myName = self.name
        otherName = 'X' if myName == 'O' else 'O'
        tree = Node(state)
        tree.expand(self)
        best_action = 0
        best_value = -10
        for node in tree.children:
            node.expand(Agent(otherName))
            for end_node in node.children:
                end_node.rnd_evaluate(myName, moves = 10)
                end_node.backpropagate()
            if node.value > best_value:
                best_action = node.action
                best_value = node.value
        return best_action
        

In [55]:
agents = (MCTS('O'), Human('X'))
game = Game(agents)
value = game.play()


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . . .

Select the column you want to insert your tile to

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . . .
. O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . . .
. O . O . . .

Select the column you want to insert your tile to

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . . .
X O . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . . .
X O . O O . .

Select the column you want to insert your tile to

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . . .
X O X O O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . O . .
X O X O O . .

Select the column you want to insert your tile to

0

## *Optional exercise: Dynamic Programming*
Then use dynamic programming to make your AI more efficient. You can use the class below (or not)

In [ ]:
class TranspositionTable:
    def __init__(self, size=1_000_000):
        self.size = size
        self.vals = [None] * size

    def board_str(self, state: State):
        return ''.join([''.join(c) for c in state.board])

    def put(self, state: State, utility: float):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        self.vals[idx] = (bstr, utility)

    def get(self, state: State):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        stored = self.vals[idx]
        if stored is None:
            return None
        if stored[0] == bstr:
            return stored[1]
        else:
            return None